In [ ]:
import rospy
from sensor_msgs.msg import PointCloud2, Image, PointField
import sensor_msgs.point_cloud2 as pc2
from std_msgs.msg import Header
from velodyne_msgs.msg import VelodyneScan
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
from cv_bridge import CvBridge
import time
import cv2
import open3d as o3d
import math

In [ ]:
ZED_TOPIC = "/islam/zed_pts"
VLP_TOPIC = "/islam/vlp_pts"
ZED_DEPTH = '/zed/zed_node/depth/depth_registered'

In [ ]:
ZED_V = 360
ZED_H = 640
ZED_H_ANGLE = 90
ZED_V_ANGLE = 60

LiDAR_V = 16
LiDAR_ANGLE = 32  # 2 degree precaution

row_theta_step = 2
row_theta_map = ZED_V/(2 * LiDAR_V)

col_phi_map = ZED_H/ZED_H_ANGLE
col_phi_max = ZED_H_ANGLE/2

m = ZED_V//(2 * LiDAR_V)

In [ ]:
rospy.init_node('sf', anonymous=True)
# zed_s = rospy.wait_for_message(ZED_TOPIC, PointCloud2)
# vlp_s = rospy.Subscriber(VLP_TOPIC, PointCloud2, vlp_callback)

In [ ]:
def sph_to_cart_pts(pts):
    pts[:,1] = cp.radians(pts[:,1])
    pts[:,2] = cp.radians(pts[:,2])

    # Convert spherical coordinates to Cartesian coordinates
    x = pts[:, 0] * cp.cos(pts[:, 1]) * cp.cos(pts[:, 2])
    y = pts[:, 0] * cp.cos(pts[:, 1]) * cp.sin(pts[:, 2])
    z = pts[:, 0] * cp.sin(pts[:, 1])
    
    return cp.asarray([x, y, z]).T

def cart_to_sph_pts(pts):
    # Convert to CuPy array
    pts = cp.asarray(pts)

    # Convert to spherical coordinates
    x, y, z = pts[:, 0], pts[:, 1], pts[:, 2]
    r = cp.sqrt(x**2 + y**2 + z**2)
    theta = cp.arctan(z / cp.sqrt(x**2 + y**2))
    phi = cp.arctan(y / x)

    return cp.column_stack((r, cp.degrees(theta), cp.degrees(phi)))
    # return appendSpherical_np(pts)[:,3:6]

def msg2pts(msg):
    return cp.array(list(pc2.read_points(msg, field_names=("x", "y", "z"))))

In [ ]:
# zed_pts = rospy.wait_for_message(ZED_TOPIC, PointCloud2)
zed_img = rospy.wait_for_message(ZED_DEPTH, Image)
vlp_pts = rospy.wait_for_message(VLP_TOPIC, PointCloud2)
if zed_img and vlp_pts:
    print('OK')

In [ ]:
bridge = CvBridge()
zed_depth = cp.array(bridge.imgmsg_to_cv2(zed_img, "32FC1"))

In [ ]:
vlp_pts = msg2pts(vlp_pts)
vlp_sph_pts_raw = cart_to_sph_pts(vlp_pts[vlp_pts[:,0] > 0])
mask = (vlp_sph_pts_raw[:, 2] < ZED_H_ANGLE/2) & (vlp_sph_pts_raw[:, 2] > -ZED_H_ANGLE/2)
vlp_sph_pts = vlp_sph_pts_raw[mask]

In [ ]:
print(np.nanmean(zed_depth[10]))

In [ ]:
vlp_depth = cp.zeros(shape=(LiDAR_V, ZED_H))

theta_max = vlp_sph_pts[:,1].max()
theta_min = vlp_sph_pts[:,1].min()
theta_range = theta_max - theta_min

for i in range(0, LiDAR_V):
    mask = (vlp_sph_pts[:, 1] < theta_max - i*theta_range/LiDAR_V) & (vlp_sph_pts[:, 1] > theta_max - (i+1)*theta_range/LiDAR_V)
    row = vlp_sph_pts[mask]
    
    cols = []
    for col in row:
        px = math.floor(ZED_H - float(col[2] + ZED_H_ANGLE/2) * ZED_H / ZED_H_ANGLE)
        cols.append(px)
        vlp_depth[i, px - 1]= col[0]


In [ ]:
plt.figure(figsize = (20,10))
plt.imshow(vlp_depth.get())
vlp_depth.shape

In [ ]:
plt.figure(figsize = (20,10))
plt.imshow(zed_depth.get())
zed_depth.shape

In [ ]:
nan_mask = cp.isnan(zed_depth)
zed_depth[nan_mask] = cp.mean(vlp_pts)
plt.figure(figsize = (20,10))
plt.imshow(zed_depth.get())
zed_depth.shape

In [ ]:
def lpf(img, ncutoff):
    # Apply 2D FFT to the image
    f = cp.fft.fft2(img)

    # Shift the zero frequency component to the center of the spectrum
    fshift = cp.fft.fftshift(f)

    # Create a circular mask of the same size as the spectrum
    rows, cols = img.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), np.uint8)
    cutoff = int(min(crow, ccol)*ncutoff)
    cv2.circle(mask, (ccol, crow), cutoff, 1, -1)
    # cv2.ellipse(mask, (ccol, crow), (1, 2) * cutoff, 0, 0, 360,  1, -1)

    mask = cp.asarray(mask)

    # Apply the mask to the shifted spectrum
    fshift_filtered = fshift * mask

    # Shift the zero frequency component back to the corner of the spectrum
    f_filtered = cp.fft.ifftshift(fshift_filtered)

    # Apply the inverse 2D FFT to the filtered spectrum
    img_filtered = cp.fft.ifft2(f_filtered)
    img_filtered = cp.real(img_filtered)

    return img_filtered


def pg(input, us_rate, ncutoff, threshold=100):
    ncutoff = ncutoff / 10
    filtered = input
    
    mask = (input[::us_rate, :] != 0)

    while threshold > 0:
        filtered = lpf(filtered, ncutoff)
        filtered[mask] = input[mask]

        threshold -= 1

    return filtered

In [ ]:
pg_depth_init = zed_depth.copy()
pg_depth_init[ZED_V//4:3*ZED_V//4-10:m, :] = vlp_depth

In [ ]:
pg_depth_init.get()

In [ ]:
plt.imshow(pg_depth_init.get())
plt.show()

In [ ]:
start = time.time()
pg_depth = pg(pg_depth_init, m, ncutoff=5, threshold=200)
print(time.time() - start)

In [ ]:
plt.imshow(pg_depth_init[::m,:].get())

In [ ]:
pg_depth.get()

In [ ]:
plt.imshow(pg_depth.get())
plt.show()

In [ ]:
def depth_to_sph_pts(depth):
    # Make sure input depth array is a CuPy array
    depth = cp.array(depth)
    
    # get the shape of the input array
    m, n = depth.shape
    azimuth_const = ZED_H_ANGLE/n
    polar_const = ZED_V_ANGLE/m

    # Create a grid of row and col indices
    row_indices, col_indices = cp.meshgrid(cp.arange(m), cp.arange(n), indexing='ij')

    # Calculate polar and azimuth angles
    polar_angles = row_indices * polar_const
    azimuth_angles = col_indices * azimuth_const

    # Stack the depth, polar_angles, and azimuth_angles along the last dimension
    pts = cp.stack((depth, polar_angles, azimuth_angles), axis=-1)

    # Reshape the pts array to the desired output shape (m * n, 3)
    pts = pts.reshape(m * n, 3)

    return pts

In [ ]:
pg_pts = sph_to_cart_pts(depth_to_sph_pts(pg_depth))

pcd_pg_lidar = o3d.geometry.PointCloud()
pcd_pg_lidar.points = o3d.utility.Vector3dVector(pg_pts.get())

In [ ]:
o3d.visualization.draw_geometries([
    pcd_pg_lidar, 
    # pcd_inp_lidar
    ])